In [30]:
import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch

from microgpt.model import MicroGPTConfig, MicroGPT
from hiq.vis import print_model

In [ ]:
# -----------------------------------------------------------------------------
# I/O
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval

# model
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 100 # how many steps to warm up for
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster

# Output directory for training checkpoints
out_dir = "out"  # Default output directory

# Sampling configuration
# This out_dir is also used by sample.py for loading trained models
eval_interval = 250  # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10  # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False
dataset = "shakespeare_char"
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256  # context of up to 256 previous characters

# microGPT setup
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3  # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000  # make equal to max_iters usually
min_lr = 1e-4  # learning_rate / 10 usually

In [32]:

# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

In [33]:

# various inits, derived attributes, I/O setup
# single GPU setup
master_process = True
seed_offset = 0
ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on matmul
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader - data_dir will be set in main() after config is loaded
def get_batch(split, data_dir):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

tokens per iteration will be: 16,384


In [34]:

# init these up here
iter_num = 0
best_val_loss = 1e9

# model init - always initialize a new model from scratch
print("Initializing a new model from scratch")
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=50304, dropout=dropout) # start with model_args from command line
gptconf = MicroGPTConfig(**model_args)
model = MicroGPT(gptconf)

# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.amp.GradScaler('cuda', enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

print_model(model)
# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

Initializing a new model from scratch
number of parameters: 29.94M
num decayed parameter tensors: 26, with 30,031,872 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters
using fused AdamW: True


🌳 MicroGPT<all params:30036864>
├── ModuleDict(transformer)
│   ├── Embedding(wte)|weight[50304,384]
│   ├── Embedding(wpe)|weight[256,384]
│   ├── ModuleList(h)
│   │   └── 💠 ]8;id=919686;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\Block]8;;\]8;id=152329;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(0-5)<🦜:1770240x6>]8;;\
│   │       ┣━━ 💠 ]8;id=129072;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\LayerNorm]8;;\]8;id=916368;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(ln_1,ln_2)<🦜:384x2>|]8;;\]8;id=895704;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\weight[384]]8;;\
│   │       ┣━━ CausalSelfAttention(attn)
│   │       ┃   ┣━━ Linear(c_attn)|weight[1152,384]
│   │       ┃   ┗━━ Linear(c_proj)|weight[384,384]
│   │       ┗━━ MLP(mlp)
│   │           ┣━━ Linear(c_fc)|weight[1536,384]
│   │           ┗━━ Linear(c_proj)|weight[384,1536]
│   └── LayerNorm(ln_f)|weight[384]
└── Linear(lm_head)|weight[50304,384]

compiling the model... (takes a ~minute)


In [36]:

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss(data_dir):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, data_dir)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [37]:
# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

In [38]:
def main():
    global iter_num, best_val_loss, model, optimizer
    
    # training loop
    # data_dir will be set here after config is loaded
    data_dir = os.path.join('data', dataset)
    
    # attempt to derive vocab_size from the dataset
    meta_path = os.path.join(data_dir, 'meta.pkl')
    meta_vocab_size = None
    if os.path.exists(meta_path):
        with open(meta_path, 'rb') as f:
            meta = pickle.load(f)
        meta_vocab_size = meta['vocab_size']
        print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")
    
    # update model_args with vocab_size if found
    if meta_vocab_size is not None:
        model_args['vocab_size'] = meta_vocab_size
        # recreate the model with the correct vocab_size
        gptconf = MicroGPTConfig(**model_args)
        model = MicroGPT(gptconf)
        model.to(device)
        # recompile if needed
        if compile:
            print("recompiling the model with updated vocab_size...")
            unoptimized_model = model
            model = torch.compile(model)
        # reconfigure optimizer
        optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
    
    # Print the final model structure after all potential modifications
    from hiq.vis import print_model
    print_model(model)
    
    X, Y = get_batch('train', data_dir) # fetch the very first batch
    t0 = time.time()
    local_iter_num = 0 # number of iterations in the lifetime of this process
    raw_model = model # no DDP wrapper needed
    running_mfu = -1.0
    while True:

        # determine and set the learning rate for this iteration
        lr = get_lr(iter_num) if decay_lr else learning_rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # evaluate the loss on train/val sets and write checkpoints
        if iter_num % eval_interval == 0 and master_process:
            losses = estimate_loss(data_dir)
            print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

            if losses['val'] < best_val_loss or always_save_checkpoint:
                best_val_loss = losses['val']
                if iter_num > 0:
                    checkpoint = {
                        'model': raw_model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'model_args': model_args,
                        'iter_num': iter_num,
                        'best_val_loss': best_val_loss,
                        'config': config,
                    }
                    print(f"saving checkpoint to {out_dir}")
                    torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
        if iter_num == 0 and eval_only:
            break

        # forward backward update, with optional gradient accumulation to simulate larger batch size
        # and using the GradScaler if data type is float16
        for micro_step in range(gradient_accumulation_steps):
            with ctx:
                logits, loss = model(X, Y)
                loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
            # immediately async prefetch next batch while model is doing the forward pass on the GPU
            X, Y = get_batch('train', data_dir)
            # backward pass, with gradient scaling if training in fp16
            scaler.scale(loss).backward()
        # clip the gradient
        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        # step the optimizer and scaler if training in fp16
        scaler.step(optimizer)
        scaler.update()
        # flush the gradients as soon as we can, no need for this memory anymore
        optimizer.zero_grad(set_to_none=True)

        # timing and logging
        t1 = time.time()
        dt = t1 - t0
        t0 = t1
        if iter_num % log_interval == 0 and master_process:
            # get loss as float. note: this is a CPU-GPU sync point
            # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
            lossf = loss.item() * gradient_accumulation_steps
            if local_iter_num >= 5: # let the training loop settle a bit
                mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
                running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
            print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
        iter_num += 1
        local_iter_num += 1

        # termination conditions
        if iter_num > max_iters:
            break

In [39]:
!python -m microgpt.prepare_dataset

Saving processed data to: /home/wukong/git.repo/pypi/microgpt/notebook/data/shakespeare_char
length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [40]:
main()

found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)


number of parameters: 10.65M
recompiling the model with updated vocab_size...
num decayed parameter tensors: 26, with 10,740,096 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters
using fused AdamW: True


🌳 OptimizedModule<all params:10745088>
└── MicroGPT(_orig_mod)
    ├── ModuleDict(transformer)
    │   ├── Embedding(wte)|weight[65,384]
    │   ├── Embedding(wpe)|weight[256,384]
    │   ├── ModuleList(h)
    │   │   └── 💠 ]8;id=138392;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\Block]8;;\]8;id=730366;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(0-5)<🦜:1770240x6>]8;;\
    │   │       ┣━━ 💠 ]8;id=444550;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\LayerNorm]8;;\]8;id=206309;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(ln_1,ln_2)<🦜:384x2>|]8;;\]8;id=827623;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\weight[384]]8;;\
    │   │       ┣━━ CausalSelfAttention(attn)
    │   │       ┃   ┣━━ Linear(c_attn)|weight[1152,384]
    │   │       ┃   ┗━━ Linear(c_proj)|weight[384,384]
    │   │       ┗━━ MLP(mlp)
    │   │           ┣━━ Linear(c_fc)|weight[1536,384]
    │   │           ┗━━ Linear(c_proj)|weight[384,1536]
    │   └── LayerNorm(ln_f)|weight[384]
    └── Linear(lm_head)|weight[65,384]

step 0: train loss 4.3128, val loss 4.3080
iter 0: loss 4.2885, time 2146.30ms, mfu -100.00%
iter 10: loss 3.2350, time 15.21ms, mfu 24.49%
iter 20: loss 2.8015, time 15.19ms, mfu 24.50%
iter 30: loss 2.6265, time 14.97ms, mfu 24.54%
iter 40: loss 2.5540, time 15.31ms, mfu 24.52%
iter 50: loss 2.5377, time 15.06ms, mfu 24.54%
iter 60: loss 2.5120, time 15.62ms, mfu 24.47%
iter 70: loss 2.5035, time 14.83ms, mfu 24.54%
iter 80: loss 2.4890, time 14.26ms, mfu 24.70%
iter 90: loss 2.4903, time 14.90ms, mfu 24.73%
iter 100: loss 2.4622, time 14.69ms, mfu 24.79%
iter 110: loss 2.4527, time 15.16ms, mfu 24.77%
iter 120: loss 2.4160, time 14.97ms, mfu 24.78%
iter 130: loss 2.4440, time 15.39ms, mfu 24.72%
iter 140: loss 2.4301, time 15.06ms, mfu 24.73%
iter 150: loss 2.3881, time 14.46ms, mfu 24.83%
iter 160: loss 2.3564, time 14.73ms, mfu 24.88%
iter 170: loss 2.3446, time 15.04ms, mfu 24.87%
iter 180: loss 2.3481, time 14.85ms, mfu 24.89%
iter 190: loss 2.3067, time 14.37ms, mfu 24.99%
iter

In [41]:
!python -m microgpt.sample

Loaded configuration from: /home/wukong/git.repo/pypi/microgpt/microgpt/pretrain/config.py
Using out_dir: out
Loading checkpoint from: out/ckpt.pt
number of parameters: 10.65M
Using data directory meta.pkl from: data/shakespeare_char/meta.pkl
Loading meta from data/shakespeare_char/meta.pkl...


ANTIGONUS:
O, well, if they could have been earned to my calamation
Which in his chief? We throw, ever crave themselves,
Would be some fit to be tastended.

CAMILLO:
Even in this:
Give me them to me no honourable. A little courtier, courage:
And in some word will may not passion with this odd
May keep the other Capitol with this increase,
We still move for us.

POLIXENES:
A son; if he were not male of all
For her by our red-full assident, what a customary
Have he straight no more stood of care,
---------------

Menenius, we are too fools and master and
severeign perfected.

CORIOLANUS:
What is the city, whom
consentents?

MENENIUS:
I cannot lack to thee ever.

CORIOLANUS:
Approceed of the evil 